# FastAPI

> ## According to the official website, FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+

Some advantages about using FastAPI with respect to other web frameworks include:

1. __It is intentionally easy to use__: The code structure and its encapsulation make it very intuitive to find the methods you are looking for in no time
2. __Fast to code__: FastAPI does lots of the hard work for you, simply add the type of request to be expected, and the corresponding response.
3. __It uses both OpenAPI and JSON__: This way, not only will your browser be able to display the data nicely, but also, the user will be able to retrieve the data efficiently.
4. __Queries are much easier to format__: Remember that lesson about type hinting? This is where it comes in extremely handy. By default your queries will have `str` format, but if you give a specific type hint in your functions, the input will be casted to that type
5. __It processes requests asynchronously__: You can send multiple requests to a FastAPI server while it is computing the operation from a previous request, so you don't have to wait for the previous request to finish to be able to accept new requests
6. __It integrates Dataclass__ decorators, which make it even easier to get your application up and running
7. __It integrates pytest__ so your API can be easily tested before being deployed.

FastAPI is relatively new (compared to Flask or Django for example), and adoption is growing faster. You might also want to check other libraries such as Tornado or Hug to create applications. However, due to its power and simplicity, we are going to focus on FastAPI.

In this lesson we will start by creating a really small API, with minimal functionality.

## Installation

We need 2 libraries:
- `fastapi` allows us to define the API. Check out the docs [here](https://fastapi.tiangolo.com)
- `uvicorn` allows fastapi to handle requests in parallel. Check out the docs [here](https://www.uvicorn.org)

In [ ]:
pip install fastapi
pip install uvicorn

# Your first API

Creating an API using FastAPI is incredibly easy, simply use a decorator on a function that will be ran in your application, and that's it:

In [1]:
import fastapi
import uvicorn

As a reminder, the structure of the address of an API is as follow:
```
REQUEST = <ROOT_URL>/<Path>?<Query Parameters>
```

Inside the `Query Parameters`, the parameters are separated by ampersand symbols (`&`). You have one example below:

<div style="text-align:center"><img src="images/API_Structure.png" width=600/></div>


Now, let's create an instance of a FastAPI.

This instance has methods which can be used to decorate functions with functionality like defining them as the function run on a `GET` request to a particular route.

> If you run the api within the notebook `uvicorn`, it will run it indefintely and you won't be able to run the proceeding cells. You should run the following code in a `.py` file.

In [1]:
%%python3
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
# The endpoint will be our localhost (127.0.0.1)
# and the port that will be listening will be 8000
# The path is '/test/calculate', but you can the path you want
# Just make sure you use the same in your browser to chck the results

@api.get('/test/calculate')
def calculate():
    return 2 + 2

uvicorn.run(api, port=8000, host='127.0.0.1') # use uvicorn to run the api so that it processes requests asynchronously

Process is interrupted.


After running the script, you can check what the API is retrieving when we visit that path (`/test/calculate`). Do this by opening [127.0.0.1:8000/test/calculate](127.0.0.1:8000/test/calculate) on your browser. You should see something like this (depending on the browser):

<div style="text-align:center"><img src="images/FastAPI_1.png" width=600/></div>



Right now, this is quite useless... Our API will simply return a simple number, and that's it... Let's spice things up adding some query parameters. You can add them simply adding arguments to the decorated function.

Query string params are passed into a request following a `?` after the path as key value pairs on either side of an `=` separated by `&`. E.g. `my/path?firstparam=1&secondparam=3`

FastAPI expects the querystring names to match the function argument names.

In [ ]:
%%python3
import fastapi
import uvicorn
api = fastapi.FastAPI() 

@api.get('/test/calculate')
def calculate(x, y):
    return x + y

uvicorn.run(api, port=8000, host='127.0.0.1')

If you visit [127.0.0.1:8000/test/calculate](127.0.0.1:8000/test/calculate) again, now your API will return a response complaining about not having enough arguments:

<div style="text-align:center"><img src="images/FastAPI_2.png" width=600/></div>

As mentioned, `x` and `y` are parameters and we need to pass them as if we were running a function. We can set them as default values and/or pass them to the URL in the form of a query:

<div style="text-align:center"><img src="images/FastAPI_3.png" width=400/></div>

Looks good!... But wait, 1 + 5 is not 15! By default the parameters of the query are read as strings. 

> You can use type hinting to change the default type of your parameters

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int): # integer type hints define the default type of the arguments, which will cast them into this type
    return x + y

uvicorn.run(api, port=8000, host='127.0.0.1')

Great, that one thing is solved. Now let's add responses characteristic of an API. Let's say that instead of adding them, we are going to divide `x` by `y` 

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

it will work fine, but try to use `y=0`... You will see Error: 500 `Internal Server Error`

<div style="text-align:center"><img src="images/FastAPI_4.png" width=400/></div>

Well, yes, the error comes from the code, but it doesn't tell the user much about the issue. We need to let them know what happened. Thus, we could send a 400 Error: `Bad Request`, meaning that the given parameters are not valid, and additionally, we could add a message. All of this will be encapsulated in the Response class from the fastapi library

In [ ]:
import fastapi
import uvicorn
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    if y == 0:
        return fastapi.Response(content='y cannot be zero', status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

One more thing you can add to Response is the data type to be returned. In many cases, APIs will return a JSON file, so we can specify the error to return the response in a JSON format.

In [ ]:
import fastapi
import uvicorn
import json
api = fastapi.FastAPI() 
 
@api.get('/test/calculate')
def calculate(x: int, y: int):
    if y == 0:
        return fastapi.Response(content=json.dumps({"error" : "y cannot be zero"}),
                                media_type='application/json',
                                status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')

Now, your response will contain a header indicating that it is in JSON format.

<div style="text-align:center"><img src="images/FastAPI_5.png" width=400/></div>

Now we've seen how to define a simple API in FastAPI and return appropriate error codes. Simple, right?

## Post requests

It's also simple to create `POST` method integrations with FastAPI using the `.post()` decorator.


In [ ]:
import fastapi
import uvicorn
import json
api = fastapi.FastAPI() 
 
@api.post('/send/data')
def process_data(x: int, y: int):
    
    if y == 0:
        return fastapi.Response(content=json.dumps({"error" : "y cannot be zero"}),
                                media_type='application/json',
                                status_code=400)
    return x / y

uvicorn.run(api, port=8000, host='127.0.0.1')



Functionally, your post requests may not require anything to be returned, but as shown in the table [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST) you should always return a `body` key in the response. If your request inserts some data into a database entry, this `body` key typically returns the resulting database entry.

# Creating your Second API

For this second API, we are going to accept the name of a celebrity, and we will return the DOB of this celebrity. Let's start setting up the code. Same as in the last notebook, we will create a very simple API and build on top of it. Once again, we will use the magic cell here, but it is not recommended. Make sure to activate the virtual environment where you installed fastapi and uvicorn!


In [ ]:
%%python
import fastapi
import uvicorn
api = fastapi.FastAPI() 

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

if __name__ == '__main__':
    uvicorn.run(api, port=8000, host='127.0.0.1')

## Docs

> FastAPI generates documentation automatically!

Run your API, then go to the `/docs` resources and you will see the API documentation. See the details in the FastAPI docs [here](https://fastapi.tiangolo.com/tutorial/body/#automatic-docs)

## CORS

Commonly, requests are made to an API from a website. That website, having a different URL, is known to be at a different "origin". By default, cross origin requests may not be allowed. This situation is known as Cross Origin Resource Sharing, or CORS. Read about how to implement CORS in the FastAPI docs [here](https://fastapi.tiangolo.com/tutorial/cors/).